### 패키지 install

In [ ]:
# 나눔고딕 설치
!apt-get update -qq
!apt-get install fonts-nanum* -qq
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 패키지 설치
!pip install kiwipiepy tomotopy flashtext -q

Selecting previously unselected package fonts-nanum.
(Reading database ... 155013 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Selecting previously unselected package fonts-nanum-eco.
Preparing to unpack .../fonts-nanum-eco_1.000-6_all.deb ...
Unpacking fonts-nanum-eco (1.000-6) ...
Selecting previously unselected package fonts-nanum-extra.
Preparing to unpack .../fonts-nanum-extra_20170925-1_all.deb ...
Unpacking fonts-nanum-extra (20170925-1) ...
Selecting previously unselected package fonts-nanum-coding.
Preparing to unpack .../fonts-nanum-coding_2.5-1_all.deb ...
Unpacking fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-extra (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Setting up fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-eco (1.000-6) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs

In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()

import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')
font_path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
import matplotlib.colors as clr

from collections import Counter
import regex
import re

from kiwipiepy import Kiwi, Option
kiwi = Kiwi()
kiwi.prepare()

from flashtext import KeywordProcessor
kp = KeywordProcessor()

import tomotopy as tp
print(tp.isa)
import sys

avx2


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: `prepare()` has no effect and will be removed in future version.


### 전처리

#### 해시태그 저장 파일 다운로드

In [ ]:
# https://drive.google.com/file/d/1lKHvngvdXcNESWL5Y7h-gDe-jRUg4QMX/view?usp=sharing
# https://drive.google.com/file/d/1ls7oDrOasdQRYlTTk5lz2qr1O1SxUJFs/view?usp=sharing
!curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=1ls7oDrOasdQRYlTTk5lz2qr1O1SxUJFs" > /dev/null
!curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1ls7oDrOasdQRYlTTk5lz2qr1O1SxUJFs" -o df_hashtag.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0    920      0 --:--:-- --:--:-- --:--:--   918
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 3378k    0 3378k    0     0  3077k      0 --:--:--  0:00:01 --:--:-- 53.0M


In [ ]:
df = pd.read_csv('df_hashtag.csv')
df

,Unnamed: 0,id,hashtags
0,0,gimdaedol,"['콜렉티브커피클럽', '리묘카페', '리묘', '리..."
1,1,by.sunset,"['레드불', '레드불화이트에디션', '회사화장실', '날씨탓', '넷..."
2,2,j.anuary_,"['오리지날스', '환절기보습', '피부진정', '티트리미스트', '스킨케어제품',..."
3,3,zzh_wzz,"['수페르가', '수페르가2750', '협찬', '미쏘', 'mixx..."
4,4,parksangchu,"['싯다르타', '뮤지컬', '위키드', '검푸른', '극단도도', '킹키부츠', ..."
...,...,...,...
227,227,jung.chelin,"['정슐랭성수', '정슐랭브런치', '정슐랭큐뮬러스', '정슐랭별셋', '정슐랭',..."
228,228,woosra,"['discovertheworld', '7wondersoftheworld', 'wo..."
229,229,kimyongcheol_0,"['은행', '용돈', '성심당', '케이크', '결기', '와이프', '딸램', ..."
230,230,muk_rim2000,"['낭쿰낭쿰', '흑돼지해물갈비전골', '치즈볶음밥', '서귀포맛집', '서귀포시청..."


In [ ]:
df['hashtags'][0]

"['콜렉티브커피클럽', '리묘카페', '리묘', '리묘', '아버지', '서울나들이', '카페우즈', '3flobby', '서울나들이', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음', '없음'

In [ ]:
#없음 지우기
for i in range(0, 231):
  df['hashtags'][i] = df['hashtags'][i].replace('없음', '')

df


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Unnamed: 0,id,hashtags
0,0,gimdaedol,"['콜렉티브커피클럽', '리묘카페', '리묘', '리..."
1,1,by.sunset,"['레드불', '레드불화이트에디션', '회사화장실', '날씨탓', '넷..."
2,2,j.anuary_,"['오리지날스', '환절기보습', '피부진정', '티트리미스트', '스킨케어제품',..."
3,3,zzh_wzz,"['수페르가', '수페르가2750', '협찬', '미쏘', 'mixx..."
4,4,parksangchu,"['싯다르타', '뮤지컬', '위키드', '검푸른', '극단도도', '킹키부츠', ..."
...,...,...,...
227,227,jung.chelin,"['정슐랭성수', '정슐랭브런치', '정슐랭큐뮬러스', '정슐랭별셋', '정슐랭',..."
228,228,woosra,"['discovertheworld', '7wondersoftheworld', 'wo..."
229,229,kimyongcheol_0,"['은행', '용돈', '성심당', '케이크', '결기', '와이프', '딸램', ..."
230,230,muk_rim2000,"['낭쿰낭쿰', '흑돼지해물갈비전골', '치즈볶음밥', '서귀포맛집', '서귀포시청..."


In [ ]:
df['hashtags'][0]

"['콜렉티브커피클럽', '리묘카페', '리묘', '리묘', '아버지', '서울나들이', '카페우즈', '3flobby', '서울나들이', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 

In [ ]:
#정규식을 이용해서 글자만 남기기
import re

for i in range(0, 232):
  df['hashtags'][i] = re.sub(r"[^가-힣a-zA-Z0-9]"," ",df['hashtags'][i])

df


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Unnamed: 0,id,hashtags
0,0,gimdaedol,리묘 ...
1,1,by.sunset,레드불 레드불화이트에디션 날씨탓 넷...
2,2,j.anuary_,오리지날스 환절기보습 피부진정 티트리미스트 스킨케어제품 ...
3,3,zzh_wzz,2750 협찬 미쏘 mixx...
4,4,parksangchu,싯다르타 뮤지컬 위키드 검푸른 극단도도 킹키부츠 ...
...,...,...,...
227,227,jung.chelin,정슐랭성수 정슐랭브런치 정슐랭큐뮬러스 정슐랭별셋 정슐랭 ...
228,228,woosra,discovertheworld 7wondersoftheworld wo...
229,229,kimyongcheol_0,은행 용돈 성심당 케이크 결기 와이프 딸램 ...
230,230,muk_rim2000,낭쿰낭쿰 흑돼지해물갈비전골 치즈볶음밥 서귀포맛집 서귀포시청...


In [ ]:
df['hashtags'][0]

'                                      리묘            아버지    서울나들이    카페우즈    3flobby    서울나들이                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [ ]:
#df['hashtag']가 문자열로 바뀌어서 다시 단어별 리스트로 만들기

for i in range(0, 232):
  df['hashtags'][i] = df['hashtags'][i].split()

df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Unnamed: 0,id,hashtags
0,0,gimdaedol,"[리묘, 아버지, 서울나들이, 카페우즈, 3flobby, 서울나들이]"
1,1,by.sunset,"[레드불, 레드불화이트에디션, 날씨탓, 넷플릭스추천, bestnine2020, su..."
2,2,j.anuary_,"[오리지날스, 환절기보습, 피부진정, 티트리미스트, 스킨케어제품, 민감성피부관리, ..."
3,3,zzh_wzz,"[2750, 협찬, 미쏘, mixxo, 미쏘에센셜, 가디건, 가디건코디, 목폴라, ..."
4,4,parksangchu,"[싯다르타, 뮤지컬, 위키드, 검푸른, 극단도도, 킹키부츠, 뮤지컬, 썸씽로튼, 플..."
...,...,...,...
227,227,jung.chelin,"[정슐랭성수, 정슐랭브런치, 정슐랭큐뮬러스, 정슐랭별셋, 정슐랭, 정슐랭삼성, 정슐..."
228,228,woosra,"[discovertheworld, 7wondersoftheworld, wonderf..."
229,229,kimyongcheol_0,"[은행, 용돈, 성심당, 케이크, 결기, 와이프, 딸램, 운동하는남자, 딸바보아빠,..."
230,230,muk_rim2000,"[낭쿰낭쿰, 흑돼지해물갈비전골, 치즈볶음밥, 서귀포맛집, 서귀포시청맛집, 서귀포, ..."


### 토픽모델링

#### PTM(k =5 min_df =1, rm_top=3)

In [ ]:
PT = tp.PTModel(k=5,min_df=1,tw=tp.TermWeight.IDF,rm_top=3) # Hyperparameter 숙지 필수
#k=10 (토픽의 개수) 
#min_df(min document frequency) 최소 100개이상의 문서에서 나온 단어로 
#min_cf 단어 빈도
#rm top => 가장 많이 나온 n개 지우기 

In [ ]:
for token in tqdm(df['hashtags'].tolist()):
  PT.add_doc(token)

  0%|          | 0/232 [00:00<?, ?it/s]

In [ ]:
PT.train(0)

In [ ]:
# 한번에 20회씩 총 500회 학습
print('Num docs:', len(PT.docs), ', Vocab size:', PT.num_vocabs, ', Num words:', PT.num_words)
print('Removed top words:', PT.removed_top_words)
print('Training...', file=sys.stderr, flush=True)
for i in range(0, 500, 20):
    PT.train(20)
    print('Iteration: {}\tLog-likelihood: {}'.format(i, PT.ll_per_word))

Training...


Num docs: 232 , Vocab size: 26243 , Num words: 106516
Removed top words: ['n', '일상', '좋아요반사']
Iteration: 0	Log-likelihood: -9.185397678859042
Iteration: 20	Log-likelihood: -9.069024051071274
Iteration: 40	Log-likelihood: -9.000110244664555
Iteration: 60	Log-likelihood: -8.93320431774389
Iteration: 80	Log-likelihood: -8.884967671605114
Iteration: 100	Log-likelihood: -8.843674097320475
Iteration: 120	Log-likelihood: -8.81154017081191
Iteration: 140	Log-likelihood: -8.790806413208408
Iteration: 160	Log-likelihood: -8.76378706918839
Iteration: 180	Log-likelihood: -8.738041614678226
Iteration: 200	Log-likelihood: -8.726453466360786
Iteration: 220	Log-likelihood: -8.712127604046135
Iteration: 240	Log-likelihood: -8.694141490959387
Iteration: 260	Log-likelihood: -8.674140825753025
Iteration: 280	Log-likelihood: -8.663909070369037
Iteration: 300	Log-likelihood: -8.656599695582091
Iteration: 320	Log-likelihood: -8.650316575034184
Iteration: 340	Log-likelihood: -8.643839979370703
Iteration: 360	

In [ ]:
PT.summary()

<Basic Info>
| PTModel (current version: 0.12.2)
| 232 docs, 106516 words
| Total Vocabs: 26246, Used Vocabs: 26243
| Entropy of words: 8.64307
| Entropy of term-weighted words: 9.15599
| Removed Vocabs: n 일상 좋아요반사
|
<Training Info>
| Iterations: 500, Burn-in steps: 0
| Optimization Interval: 10
| Log-likelihood per word: -8.60540
|
<Initial Parameters>
| tw: TermWeight.IDF
| min_cf: 0 (minimum collection frequency of words)
| min_df: 1 (minimum document frequency of words)
| rm_top: 3 (the number of top words to be removed)
| k: 5 (the number of topics between 1 ~ 32767)
| p: 50 (the number of pseudo documents The default value is changed to `10 * k`.)
| alpha: [0.1] (hyperparameter of Dirichlet distribution for document-topic, given as a single `float` in case of symmetric prior and as a list with length `k` of `float` in case of asymmetric prior.)
| eta: 0.01 (hyperparameter of Dirichlet distribution for topic-word)
| seed: 251960769 (random seed)
| trained in version 0.12.2
|
<Para

#### LDA(k =5 min_df =1, rm_top=3)

In [ ]:
LDA = tp.LDAModel(k=5,min_df=1,tw=tp.TermWeight.IDF,rm_top=3) # Hyperparameter 숙지 필수
#k=10 (토픽의 개수) 
#min_df(min document frequency) 최소 100개이상의 문서에서 나온 단어로 
#min_cf 단어 빈도
#rm top => 가장 많이 나온 n개 지우기 

In [ ]:
for token in tqdm(df['hashtags'].tolist()):
  LDA.add_doc(token)

  0%|          | 0/232 [00:00<?, ?it/s]

In [ ]:
LDA.train(0)

In [ ]:
# 한번에 20회씩 총 500회 학습
print('Num docs:', len(LDA.docs), ', Vocab size:', LDA.num_vocabs, ', Num words:', LDA.num_words)
print('Removed top words:', LDA.removed_top_words)
print('Training...', file=sys.stderr, flush=True)
for i in range(0, 500, 20):
    LDA.train(20)
    print('Iteration: {}\tLog-likelihood: {}'.format(i, LDA.ll_per_word))

Training...


Num docs: 232 , Vocab size: 26243 , Num words: 106516
Removed top words: ['n', '일상', '좋아요반사']
Iteration: 0	Log-likelihood: -8.991877947960177
Iteration: 20	Log-likelihood: -8.886593647325185
Iteration: 40	Log-likelihood: -8.801015282578062
Iteration: 60	Log-likelihood: -8.739007293828319
Iteration: 80	Log-likelihood: -8.689155873582232
Iteration: 100	Log-likelihood: -8.653864095184
Iteration: 120	Log-likelihood: -8.620884381796834
Iteration: 140	Log-likelihood: -8.588533190650866
Iteration: 160	Log-likelihood: -8.572021433360408
Iteration: 180	Log-likelihood: -8.555853620536412
Iteration: 200	Log-likelihood: -8.541681798742287
Iteration: 220	Log-likelihood: -8.529389112530561
Iteration: 240	Log-likelihood: -8.522640759583467
Iteration: 260	Log-likelihood: -8.514896344363981
Iteration: 280	Log-likelihood: -8.505251761500524
Iteration: 300	Log-likelihood: -8.496162341817303
Iteration: 320	Log-likelihood: -8.491698225638034
Iteration: 340	Log-likelihood: -8.477738181326728
Iteration: 360	

In [ ]:
LDA.summary()

<Basic Info>
| LDAModel (current version: 0.12.2)
| 232 docs, 106516 words
| Total Vocabs: 26246, Used Vocabs: 26243
| Entropy of words: 8.64307
| Entropy of term-weighted words: 9.15599
| Removed Vocabs: n 일상 좋아요반사
|
<Training Info>
| Iterations: 500, Burn-in steps: 0
| Optimization Interval: 10
| Log-likelihood per word: -8.44997
|
<Initial Parameters>
| tw: TermWeight.IDF
| min_cf: 0 (minimum collection frequency of words)
| min_df: 1 (minimum document frequency of words)
| rm_top: 3 (the number of top words to be removed)
| k: 5 (the number of topics between 1 ~ 32767)
| alpha: [0.1] (hyperparameter of Dirichlet distribution for document-topic, given as a single `float` in case of symmetric prior and as a list with length `k` of `float` in case of asymmetric prior.)
| eta: 0.01 (hyperparameter of Dirichlet distribution for topic-word)
| seed: 1006888216 (random seed)
| trained in version 0.12.2
|
<Parameters>
| alpha (Dirichlet prior on the per-document topic distributions)
|  [0.0

## 파라미터 변경


#### K 값 변화 (K=10)

In [ ]:
PTM_k7 = tp.PTModel(k=10,min_df=1,tw=tp.TermWeight.IDF,rm_top=3) # Hyperparameter 숙지 필수
#k=10 (토픽의 개수) 
#min_df(min document frequency) 최소 100개이상의 문서에서 나온 단어로 
#min_cf 단어 빈도
#rm top => 가장 많이 나온 n개 지우기 

In [ ]:
for token in tqdm(df['hashtags'].tolist()):
  PTM_k7.add_doc(token)

  0%|          | 0/232 [00:00<?, ?it/s]

In [ ]:
PTM_k10.train(0)

In [ ]:
# 한번에 20회씩 총 500회 학습
print('Num docs:', len(PTM_k10.docs), ', Vocab size:', PTM_k10.num_vocabs, ', Num words:', PTM_k10.num_words)
print('Removed top words:', PTM_k10.removed_top_words)
print('Training...', file=sys.stderr, flush=True)
for i in range(0, 500, 20):
    PTM_k10.train(20)
    print('Iteration: {}\tLog-likelihood: {}'.format(i, PTM_k10.ll_per_word))

Training...


Num docs: 232 , Vocab size: 26243 , Num words: 106516
Removed top words: ['n', '일상', '좋아요반사']
Iteration: 0	Log-likelihood: -8.887792795985947
Iteration: 20	Log-likelihood: -8.706757183491552
Iteration: 40	Log-likelihood: -8.593984859722354
Iteration: 60	Log-likelihood: -8.515212624967774
Iteration: 80	Log-likelihood: -8.450484196189759
Iteration: 100	Log-likelihood: -8.390906742664827
Iteration: 120	Log-likelihood: -8.35619499121647
Iteration: 140	Log-likelihood: -8.32260019226842
Iteration: 160	Log-likelihood: -8.296851599085516
Iteration: 180	Log-likelihood: -8.264317123867974
Iteration: 200	Log-likelihood: -8.245101012339287
Iteration: 220	Log-likelihood: -8.229695820198456
Iteration: 240	Log-likelihood: -8.216705655095273
Iteration: 260	Log-likelihood: -8.19910995081242
Iteration: 280	Log-likelihood: -8.187461047023085
Iteration: 300	Log-likelihood: -8.181261348174328
Iteration: 320	Log-likelihood: -8.169055464218985
Iteration: 340	Log-likelihood: -8.162006571440601
Iteration: 360	

In [ ]:
PTM_k10.summary()

<Basic Info>
| PTModel (current version: 0.12.2)
| 232 docs, 106516 words
| Total Vocabs: 26246, Used Vocabs: 26243
| Entropy of words: 8.64307
| Entropy of term-weighted words: 9.15599
| Removed Vocabs: n 일상 좋아요반사
|
<Training Info>
| Iterations: 500, Burn-in steps: 0
| Optimization Interval: 10
| Log-likelihood per word: -8.09442
|
<Initial Parameters>
| tw: TermWeight.IDF
| min_cf: 0 (minimum collection frequency of words)
| min_df: 1 (minimum document frequency of words)
| rm_top: 3 (the number of top words to be removed)
| k: 10 (the number of topics between 1 ~ 32767)
| p: 100 (the number of pseudo documents The default value is changed to `10 * k`.)
| alpha: [0.1] (hyperparameter of Dirichlet distribution for document-topic, given as a single `float` in case of symmetric prior and as a list with length `k` of `float` in case of asymmetric prior.)
| eta: 0.01 (hyperparameter of Dirichlet distribution for topic-word)
| seed: 4197036719 (random seed)
| trained in version 0.12.2
|
<P

In [ ]:
LDA_k10 = tp.LDAModel(k=10,min_df=1,tw=tp.TermWeight.IDF,rm_top=3) # Hyperparameter 숙지 필수
#k=10 (토픽의 개수) 
#min_df(min document frequency) 최소 100개이상의 문서에서 나온 단어로 
#min_cf 단어 빈도
#rm top => 가장 많이 나온 n개 지우기 

In [ ]:
for token in tqdm(df['hashtags'].tolist()):
  LDA_k10.add_doc(token)

  0%|          | 0/232 [00:00<?, ?it/s]

In [ ]:
LDA_k10.train(0)

In [ ]:
# 한번에 20회씩 총 500회 학습
print('Num docs:', len(LDA_k10.docs), ', Vocab size:', LDA_k10.num_vocabs, ', Num words:', LDA_k10.num_words)
print('Removed top words:', LDA_k10.removed_top_words)
print('Training...', file=sys.stderr, flush=True)
for i in range(0, 500, 20):
    LDA_k10.train(20)
    print('Iteration: {}\tLog-likelihood: {}'.format(i, LDA_k10.ll_per_word))

Training...


Num docs: 232 , Vocab size: 26243 , Num words: 106516
Removed top words: ['n', '일상', '좋아요반사']
Iteration: 0	Log-likelihood: -8.666540121455778
Iteration: 20	Log-likelihood: -8.4667463510217
Iteration: 40	Log-likelihood: -8.354086081024741
Iteration: 60	Log-likelihood: -8.26549534572236
Iteration: 80	Log-likelihood: -8.199245778734085
Iteration: 100	Log-likelihood: -8.160756004500772
Iteration: 120	Log-likelihood: -8.132143880792361
Iteration: 140	Log-likelihood: -8.107130322183094
Iteration: 160	Log-likelihood: -8.079952597489829
Iteration: 180	Log-likelihood: -8.057991630122142
Iteration: 200	Log-likelihood: -8.040307067151973
Iteration: 220	Log-likelihood: -8.024074152672855
Iteration: 240	Log-likelihood: -8.013125574949367
Iteration: 260	Log-likelihood: -8.000455391603777
Iteration: 280	Log-likelihood: -7.984105718013298
Iteration: 300	Log-likelihood: -7.980805444791805
Iteration: 320	Log-likelihood: -7.974908225132263
Iteration: 340	Log-likelihood: -7.960630498936325
Iteration: 360	

In [ ]:
LDA_k10.summary()

<Basic Info>
| LDAModel (current version: 0.12.2)
| 232 docs, 106516 words
| Total Vocabs: 26246, Used Vocabs: 26243
| Entropy of words: 8.64307
| Entropy of term-weighted words: 9.15599
| Removed Vocabs: n 일상 좋아요반사
|
<Training Info>
| Iterations: 500, Burn-in steps: 0
| Optimization Interval: 10
| Log-likelihood per word: -7.91925
|
<Initial Parameters>
| tw: TermWeight.IDF
| min_cf: 0 (minimum collection frequency of words)
| min_df: 1 (minimum document frequency of words)
| rm_top: 3 (the number of top words to be removed)
| k: 10 (the number of topics between 1 ~ 32767)
| alpha: [0.1] (hyperparameter of Dirichlet distribution for document-topic, given as a single `float` in case of symmetric prior and as a list with length `k` of `float` in case of asymmetric prior.)
| eta: 0.01 (hyperparameter of Dirichlet distribution for topic-word)
| seed: 3681650542 (random seed)
| trained in version 0.12.2
|
<Parameters>
| alpha (Dirichlet prior on the per-document topic distributions)
|  [0.

#### K 값 변화(k=7)

In [ ]:
PTM_k7 = tp.PTModel(k=7,min_df=1,tw=tp.TermWeight.IDF,rm_top=3) # Hyperparameter 숙지 필수
#k=7 (토픽의 개수) 
#min_df(min document frequency) 최소 100개이상의 문서에서 나온 단어로 
#min_cf 단어 빈도
#rm top => 가장 많이 나온 n개 지우기 

In [ ]:
for token in tqdm(df['hashtags'].tolist()):
  PTM_k7.add_doc(token)

  0%|          | 0/232 [00:00<?, ?it/s]

In [ ]:
PTM_k7.train(0)

In [ ]:
# 한번에 20회씩 총 500회 학습
print('Num docs:', len(PTM_k7.docs), ', Vocab size:', PTM_k7.num_vocabs, ', Num words:', PTM_k7.num_words)
print('Removed top words:', PTM_k7.removed_top_words)
print('Training...', file=sys.stderr, flush=True)
for i in range(0, 500, 20):
    PTM_k7.train(20)
    print('Iteration: {}\tLog-likelihood: {}'.format(i, PTM_k7.ll_per_word))

Training...


Num docs: 232 , Vocab size: 26243 , Num words: 106516
Removed top words: ['n', '일상', '좋아요반사']
Iteration: 0	Log-likelihood: -9.071775349531066
Iteration: 20	Log-likelihood: -8.893238645759062
Iteration: 40	Log-likelihood: -8.800828275666852
Iteration: 60	Log-likelihood: -8.72245021711069
Iteration: 80	Log-likelihood: -8.659456720466437
Iteration: 100	Log-likelihood: -8.610872824710949
Iteration: 120	Log-likelihood: -8.564147460723404
Iteration: 140	Log-likelihood: -8.538918823054978
Iteration: 160	Log-likelihood: -8.50833261783355
Iteration: 180	Log-likelihood: -8.491472526775066
Iteration: 200	Log-likelihood: -8.478578234338151
Iteration: 220	Log-likelihood: -8.459384721976674
Iteration: 240	Log-likelihood: -8.44045327673733
Iteration: 260	Log-likelihood: -8.423096586584675
Iteration: 280	Log-likelihood: -8.419277565969155
Iteration: 300	Log-likelihood: -8.403576821984476
Iteration: 320	Log-likelihood: -8.3912500073928
Iteration: 340	Log-likelihood: -8.383745834217285
Iteration: 360	Lo

In [ ]:
PTM_k7.summary()

<Basic Info>
| PTModel (current version: 0.12.2)
| 232 docs, 106516 words
| Total Vocabs: 26246, Used Vocabs: 26243
| Entropy of words: 8.64307
| Entropy of term-weighted words: 9.15599
| Removed Vocabs: n 일상 좋아요반사
|
<Training Info>
| Iterations: 500, Burn-in steps: 0
| Optimization Interval: 10
| Log-likelihood per word: -8.35252
|
<Initial Parameters>
| tw: TermWeight.IDF
| min_cf: 0 (minimum collection frequency of words)
| min_df: 1 (minimum document frequency of words)
| rm_top: 3 (the number of top words to be removed)
| k: 7 (the number of topics between 1 ~ 32767)
| p: 70 (the number of pseudo documents The default value is changed to `10 * k`.)
| alpha: [0.1] (hyperparameter of Dirichlet distribution for document-topic, given as a single `float` in case of symmetric prior and as a list with length `k` of `float` in case of asymmetric prior.)
| eta: 0.01 (hyperparameter of Dirichlet distribution for topic-word)
| seed: 2766741777 (random seed)
| trained in version 0.12.2
|
<Par

In [ ]:
LDA_k7 = tp.LDAModel(k=7,min_df=1,tw=tp.TermWeight.IDF,rm_top=3) # Hyperparameter 숙지 필수
#k=7 (토픽의 개수) 
#min_df(min document frequency) 최소 100개이상의 문서에서 나온 단어로 
#min_cf 단어 빈도
#rm top => 가장 많이 나온 n개 지우기 

In [ ]:
for token in tqdm(df['hashtags'].tolist()):
  LDA_k7.add_doc(token)

  0%|          | 0/232 [00:00<?, ?it/s]

In [ ]:
LDA_k7.train(0)

In [ ]:
# 한번에 20회씩 총 500회 학습
print('Num docs:', len(LDA_k7.docs), ', Vocab size:', LDA_k7.num_vocabs, ', Num words:', LDA_k7.num_words)
print('Removed top words:', LDA_k7.removed_top_words)
print('Training...', file=sys.stderr, flush=True)
for i in range(0, 500, 20):
    LDA_k7.train(20)
    print('Iteration: {}\tLog-likelihood: {}'.format(i, LDA_k7.ll_per_word))

Training...


Num docs: 232 , Vocab size: 26243 , Num words: 106516
Removed top words: ['n', '일상', '좋아요반사']
Iteration: 0	Log-likelihood: -8.838680871471423
Iteration: 20	Log-likelihood: -8.68526191139141
Iteration: 40	Log-likelihood: -8.572990269792374
Iteration: 60	Log-likelihood: -8.505578753229246
Iteration: 80	Log-likelihood: -8.439341964902606
Iteration: 100	Log-likelihood: -8.39228170904552
Iteration: 120	Log-likelihood: -8.360967071508185
Iteration: 140	Log-likelihood: -8.328877494507992
Iteration: 160	Log-likelihood: -8.304721718352047
Iteration: 180	Log-likelihood: -8.283938578818283
Iteration: 200	Log-likelihood: -8.265952868510515
Iteration: 220	Log-likelihood: -8.247466419978428
Iteration: 240	Log-likelihood: -8.232893602366838
Iteration: 260	Log-likelihood: -8.217714877807634
Iteration: 280	Log-likelihood: -8.205129985280024
Iteration: 300	Log-likelihood: -8.200286611615788
Iteration: 320	Log-likelihood: -8.192960251590641
Iteration: 340	Log-likelihood: -8.18198605720833
Iteration: 360	

In [ ]:
LDA_k7.summary()

<Basic Info>
| LDAModel (current version: 0.12.2)
| 232 docs, 106516 words
| Total Vocabs: 26246, Used Vocabs: 26243
| Entropy of words: 8.64307
| Entropy of term-weighted words: 9.15599
| Removed Vocabs: n 일상 좋아요반사
|
<Training Info>
| Iterations: 500, Burn-in steps: 0
| Optimization Interval: 10
| Log-likelihood per word: -8.15546
|
<Initial Parameters>
| tw: TermWeight.IDF
| min_cf: 0 (minimum collection frequency of words)
| min_df: 1 (minimum document frequency of words)
| rm_top: 3 (the number of top words to be removed)
| k: 7 (the number of topics between 1 ~ 32767)
| alpha: [0.1] (hyperparameter of Dirichlet distribution for document-topic, given as a single `float` in case of symmetric prior and as a list with length `k` of `float` in case of asymmetric prior.)
| eta: 0.01 (hyperparameter of Dirichlet distribution for topic-word)
| seed: 3390776730 (random seed)
| trained in version 0.12.2
|
<Parameters>
| alpha (Dirichlet prior on the per-document topic distributions)
|  [0.0

#### LDA(k =5 min_df =1, rm_top=3)

In [ ]:
LDA_k5_rm_top5 = tp.LDAModel(k=5,min_df=1,tw=tp.TermWeight.IDF,rm_top=2) # Hyperparameter 숙지 필수
#k=10 (토픽의 개수) 
#min_df(min document frequency) 최소 100개이상의 문서에서 나온 단어로 
#min_cf 단어 빈도
#rm top => 가장 많이 나온 n개 지우기 

In [ ]:
for token in tqdm(df['hashtags'].tolist()):
  LDA_k5_rm_top5.add_doc(token)

  0%|          | 0/232 [00:00<?, ?it/s]

In [ ]:
LDA_k5_rm_top5.train(0)

In [ ]:
# 한번에 20회씩 총 500회 학습
print('Num docs:', len(LDA_k5_rm_top5.docs), ', Vocab size:', LDA_k5_rm_top5.num_vocabs, ', Num words:', LDA_k5_rm_top5.num_words)
print('Removed top words:', LDA_k5_rm_top5.removed_top_words)
print('Training...', file=sys.stderr, flush=True)
for i in range(0, 500, 20):
    LDA_k5_rm_top5.train(20)
    print('Iteration: {}\tLog-likelihood: {}'.format(i, LDA_k5_rm_top5.ll_per_word))

Training...


Num docs: 232 , Vocab size: 26244 , Num words: 107338
Removed top words: ['n', '일상']
Iteration: 0	Log-likelihood: -8.954945127803908
Iteration: 20	Log-likelihood: -8.835601586254572
Iteration: 40	Log-likelihood: -8.760496928972618
Iteration: 60	Log-likelihood: -8.690188712527442
Iteration: 80	Log-likelihood: -8.632068030191222
Iteration: 100	Log-likelihood: -8.58873421636442
Iteration: 120	Log-likelihood: -8.553133548983268
Iteration: 140	Log-likelihood: -8.528422740520897
Iteration: 160	Log-likelihood: -8.50481611898656
Iteration: 180	Log-likelihood: -8.482904411610168
Iteration: 200	Log-likelihood: -8.471399076211277
Iteration: 220	Log-likelihood: -8.453664957758024
Iteration: 240	Log-likelihood: -8.447272718381562
Iteration: 260	Log-likelihood: -8.439154536344123
Iteration: 280	Log-likelihood: -8.432312149071558
Iteration: 300	Log-likelihood: -8.420566149088716
Iteration: 320	Log-likelihood: -8.415800036882532
Iteration: 340	Log-likelihood: -8.40853197988967
Iteration: 360	Log-likel

In [ ]:
LDA_k5_rm_top5.summary()

<Basic Info>
| LDAModel (current version: 0.12.2)
| 232 docs, 107338 words
| Total Vocabs: 26246, Used Vocabs: 26244
| Entropy of words: 8.62182
| Entropy of term-weighted words: 9.14905
| Removed Vocabs: n 일상
|
<Training Info>
| Iterations: 500, Burn-in steps: 0
| Optimization Interval: 10
| Log-likelihood per word: -8.37549
|
<Initial Parameters>
| tw: TermWeight.IDF
| min_cf: 0 (minimum collection frequency of words)
| min_df: 1 (minimum document frequency of words)
| rm_top: 2 (the number of top words to be removed)
| k: 5 (the number of topics between 1 ~ 32767)
| alpha: [0.1] (hyperparameter of Dirichlet distribution for document-topic, given as a single `float` in case of symmetric prior and as a list with length `k` of `float` in case of asymmetric prior.)
| eta: 0.01 (hyperparameter of Dirichlet distribution for topic-word)
| seed: 1036267303 (random seed)
| trained in version 0.12.2
|
<Parameters>
| alpha (Dirichlet prior on the per-document topic distributions)
|  [0.1449564